In [1]:
# Mount Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import necessary libraries

import pandas as pd
import numpy as np

In [16]:
# Read in CSVs from unpacked json files

roster = pd.read_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/rosters.csv')
engagement = pd.read_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/nextDayPlayerEngagement.csv')
games = pd.read_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/games.csv')
playerBoxScore = pd.read_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/playerBoxScores.csv')
standings = pd.read_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/standings.csv')

roster.columns

Index(['Unnamed: 0', 'playerId', 'gameDate', 'teamId', 'statusCode', 'status'], dtype='object')

In [8]:
# Select players from Cleveland Guardians (teamId = 114)

guardians_id = 114

playerIDs = roster[roster['teamId']==guardians_id]['playerId']
playerIDs = (np.array(playerIDs))
playerIDs = np.unique(playerIDs)

In [9]:
# Filter CSVs to games in which the Guardians were playing and filter columns to all that is necessary

engagement = engagement[engagement['playerId'].isin(playerIDs)== True]
engagement = engagement[['engagementMetricsDate','playerId','target1','target2','target3','target4']]
games= games[(games['awayId']==guardians_id) | (games['homeId']==guardians_id)]
games = games[['gameDate','gamePk','dayNight','homeId','awayId','homeWins','awayWins','homeLosses','awayLosses','homeWinPct','awayWinPct','homeScore','awayScore','homeWinner','awayWinner']]
playerBoxScore = playerBoxScore[playerBoxScore['teamId']==guardians_id]
playerBoxScore = playerBoxScore[['playerId','gamePk','gameDate','positionName','runsScored','doubles','triples','homeRuns','strikeOuts','hits','atBats','caughtStealing','stolenBases','totalBases','rbi','runsPitching','homeRunsPitching','strikeOutsPitching','hitsPitching','inningsPitched','strikes','saves','assists','errors','chances']]
standings = standings[standings['teamId']==guardians_id]
standings = standings[['gameDate','streakCode','pct','runsAllowed','runsScored']]

In [10]:
# Create identical column in all datasets to use as merge column and use as index

engagement['nextday'] = pd.to_datetime(engagement['engagementMetricsDate'])
games['nextday'] = pd.to_datetime(games['gameDate']) + pd.DateOffset(1)
playerBoxScore['nextday'] = pd.to_datetime(playerBoxScore['gameDate']) + pd.DateOffset(1)
standings['nextday'] = pd.to_datetime(standings['gameDate']) + pd.DateOffset(1)

In [11]:
# Set index for all datasets

engagement.set_index('nextday', inplace=True)
games.set_index('nextday', inplace=True)
playerBoxScore.set_index('nextday', inplace=True)
standings.set_index('nextday', inplace=True)

In [12]:
# Create subsets for each dataset based on begining and end dates of four seasons we have data for

engagement1 = engagement.loc['2018-03-30':'2018-10-29'] 
engagement2 = engagement.loc['2019-03-21':'2019-10-31'] 
engagement3 = engagement.loc['2020-07-24':'2020-10-28'] 
engagement4 = engagement.loc['2021-04-02':'2021-05-01']

games1 = games.loc['2018-03-30':'2018-10-29']
games2 = games.loc['2019-03-21':'2019-10-31'] 
games3 = games.loc['2020-07-24':'2020-10-28'] 
games4 = games.loc['2021-04-02':'2021-05-01']

playerBox1 = playerBoxScore.loc['2018-03-30':'2018-10-29'] 
playerBox2 = playerBoxScore.loc['2019-03-21':'2019-10-31'] 
playerBox3 = playerBoxScore.loc['2020-07-24':'2020-10-28'] 
playerBox4 = playerBoxScore.loc['2021-04-02':'2021-05-01']

standings1 = standings.loc['2018-03-30':'2018-10-29']
standings2 = standings.loc['2019-03-21':'2019-10-31'] 
standings3 =  standings.loc['2020-07-24':'2020-10-28'] 
standings4 = standings.loc['2021-04-02':'2021-05-01']

In [13]:
# Merge all datasets by year

merge1y1 = pd.merge(engagement1, playerBox1, how='left', on=['playerId','nextday'])
merge2y1 = pd.merge(merge1y1, games1, how='left',on=['nextday','gamePk'])
merge3y1 = pd.merge(merge2y1, standings1 , how='left', on='nextday')

merge1y2 = pd.merge(engagement2, playerBox2, how='left', on=['playerId','nextday'])
merge2y2 = pd.merge(merge1y2, games2, how='left',on=['nextday','gamePk'])
merge3y2 = pd.merge(merge2y2, standings2 , how='left', on='nextday')

merge1y3 = pd.merge(engagement3, playerBox3, how='left', on=['playerId','nextday'])
merge2y3 = pd.merge(merge1y3, games3, how='left',on=['nextday','gamePk'])
merge3y3 = pd.merge(merge2y3, standings3 , how='left', on='nextday')

merge1y4 = pd.merge(engagement4, playerBox4, how='left', on=['playerId','nextday'])
merge2y4 = pd.merge(merge1y4, games4, how='left',on=['nextday','gamePk'])
merge3y4 = pd.merge(merge2y4, standings4 , how='left', on='nextday')

In [15]:
# Concatenate all datasets sequentially by year

dfs = [merge3y1, merge3y2, merge3y3, merge3y4]

final_df = pd.concat(dfs)

In [17]:
# Write dataframe to CSV

final_df.to_csv('/content/drive/MyDrive/IndependentProjects/GuardiansProject/CSVs/guardians_engagement.csv')